In [57]:
import praw

client_secret="5LSSWHnkHVM5-BIkxpgYlu8fKIqnzw"
client_id = "e8O8uDpRkFbES6y2ZA9XwQ"

user_agent = "My Big Data Project"

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

In [58]:
def search_reddit(keyword, num_posts=10):
    subreddit = reddit.subreddit(keyword)

    # Get the top hot messages 
    messages = subreddit.hot(limit=num_posts)

    data = []

    # Loop through each message and access its data 
    for message in messages:
        data.append(message.title)
    
    return data


In [59]:
#Scrapping for keywords
keywords = ["india","ipl", "space", "sleep", "earth"]

#for each keyword scrapping the top 10 reddits of current time

all_word_data = []

for word in keywords :
    scrap_data = search_reddit(word, 10)
    all_word_data.append(scrap_data)
    print("Word : ", word)
    print("Data : ",  scrap_data)
    print()

Word :  india
Data :  ['Ask India Thread', 'Mental & Emotional Health Support Thread', "'98% of Currency Came Back, So Where Was Black Money Eradication?': Supreme Court Justice BV Nagarathna on Demonetisation", 'American Sikhs in California vote on independence from India', "Over 770 kg of explosives seized from local RSS leader's home in Kerala weeks before polls", 'First Rs.20 coin of India', "Help me get out of scarcity mindset, it's taking a toll", 'Magunta Reddy, whose son turned approver in excise case against Kejriwal, gets ticket from BJP ally TDP', 'Where do you see the country going after Modi is gone? ', 'Uttar Pradesh: People in large numbers take part in the funeral procession of gangster-turned-politician Mukhtar Ansari in Mohammadabad, Ghazipur']

Word :  ipl
Data :  ["🚀 IPL 2024: Dive Into Our Live Chat for Instant Updates & Inside Scoops on Every Match, Team, and Player! 💬🔥 Don't Miss Out!", 'Match Thread: 10th Match - Royal Challengers Bengaluru vs Kolkata Knight Rid

In [60]:
import spacy
import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')

In [61]:
# Load the pre-trained English language model with medium-sized word vectors
nlp = spacy.load("en_core_web_md")

In [62]:
def reddit_to_vector(reddit):
    # Process the email with spaCy
    doc = nlp(reddit)

    # Extract word vectors and calculate the average vector
    word_vectors = [token.vector for token in doc if token.has_vector]
    
    if word_vectors:
        avg_vector = np.mean(word_vectors, axis=0)
        return avg_vector
    else:
        # If no word vectors are found, return a zero vector
        return np.zeros(nlp.vocab.vectors.shape[1])

In [63]:
def process_reddits(data):
    vectors = []
    labels = []

    for reddit in data :
        reddit = str(reddit)
        vector = reddit_to_vector(reddit)
        
        vectors.append(vector)
    
    return np.array(vectors)

In [64]:
#convert each reddit into vector form :- 
all_word_processed_data = []
for single_word_data in all_word_data :
    processed_data = process_reddits(single_word_data)
    all_word_processed_data.append(processed_data)

In [65]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

In [71]:
num_clusters = 2

kmeans = KMeans(n_clusters=num_clusters)

In [72]:
#making cluster for each word
for data in all_word_processed_data :
    distance_matrix = 1 - cosine_similarity(data)
    kmeans.fit(distance_matrix)
    cluster_labels = kmeans.labels_
    print(cluster_labels)

[1 1 0 0 0 0 0 0 0 0]
[1 1 0 0 0 0 0 0 1 0]
[0 1 1 0 0 0 1 0 0 1]
[0 1 1 1 1 0 0 1 1 1]
[0 1 1 0 1 1 0 0 1 0]
